In [ ]:
from source.class_yolo_trainer import YOLOTrainer

dataset_constructor = YOLOTrainer
dataset_constructor = YOLOTrainer()

In [ ]:
dataset_constructor.image_folder = "pelotas_12-08/images"
dataset_constructor.annotations_folder = "pelotas_12-08/labels"
dataset_constructor.yolo_Classes = "pelotas_12-08/classes.txt"
dataset_constructor.yolo_Notes = "pelotas_12-08/notes.json"
dataset_constructor.test_percentual_divisor = 20
dataset_constructor.slicing_dataset_for_traning()

In [ ]:
dataset_constructor.training_YOLO_model()

In [ ]:
dataset_constructor.object_to_predict = "2024-08-08_00.22.40.png"
dataset_constructor.predict_confidence = 0.5
dataset_constructor.predict_YOLO_model("train2")

In [ ]:
from utils.check_file import check_file

check_file("any_file")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("pelotas_12-08", "pelotas_12-08")

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

# Run batched inference on a list of images
results = model("vid_1.mp4")  # return a generator of Results objects
# Process results generator
i = 0
for result in results:
    device = result.cuda()
    boxes = result.boxes  # Boxes object for bounding box outputs
    conf = boxes.conf
    cls = boxes.cls
    fmt = boxes.xywhn
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    # result.show()  # display to screen
    result.save(filename=f"result{i}.jpg")  # save to disk
    i += 1